In [1]:
import os
import re

import theano

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing

Using gpu device 0: GeForce GTX 980 (CNMeM is disabled)


In [2]:
train = pd.read_json('../data/train.json/train.json')
train.head()

,cuisine,id,ingredients
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes..."
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g..."
2,filipino,20130,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,indian,22213,"[water, vegetable oil, wheat, salt]"
4,indian,13162,"[black pepper, shallots, cornflour, cayenne pe..."


In [3]:
vectorizer = CountVectorizer(max_features = 2000)
ingredients = train['ingredients']
words_list = [' '.join(x) for x in ingredients]

#Make label encoder
le = preprocessing.LabelEncoder()
le.fit(train["cuisine"])

print le.transform(train["cuisine"])[0:100] 

[ 6 16  4  7  7 10 17  9 13  9  9  3  9 13  9  7  1  9 18 19 18 13 16  3  9
  3  2  9  3 13  9  2 13 18  9  2  9  4 16 16  9  0 13  7 13  3  5 16 16 16
 11 16  9 16  9 10 11  7  9  8 18 18  7 10  9 18 12  5  5 16 17  7 14  9  9
 14 14 19 11 13  2 16  5  7  7  9  9  7 12 17  9 16 16  6 13 13 16  7  9  9]


In [4]:
#create a bag of words and convert to a array and then print the shape
bag_of_words = vectorizer.fit(words_list)
bag_of_words = vectorizer.transform(words_list).toarray()
print(bag_of_words.shape)

vectorizertfidf = TfidfVectorizer(min_df=1)
tfidf = vectorizertfidf.fit_transform(words_list).toarray()
print tfidf.shape

(39774, 2000)
(39774, 3010)


/usr/local/lib/python2.7/dist-packages/numpy/core/fromnumeric.py:2641: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


In [5]:
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils

Using Theano backend.


In [8]:
batch_size = 128
nb_epoch = 15

X_train = tfidf
y_train = le.transform(train["cuisine"])

nb_classes = len(le.classes_)

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)

#output_dim=64, input_dim=100,
model = Sequential()
model.add(Dense(1024,input_shape=(3010,)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(.4))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

rms = RMSprop()
model.compile(loss='categorical_crossentropy', optimizer=rms)

model.fit(X_train, Y_train,
          batch_size=batch_size, nb_epoch=nb_epoch,
          show_accuracy=True, validation_split=.1, verbose=2)

Train on 35796 samples, validate on 3978 samples
Epoch 1/15
4s - loss: 1.5009 - acc: 0.5581 - val_loss: 0.9240 - val_acc: 0.7205
Epoch 2/15
4s - loss: 0.8550 - acc: 0.7429 - val_loss: 0.7653 - val_acc: 0.7672
Epoch 3/15
4s - loss: 0.7009 - acc: 0.7887 - val_loss: 0.7013 - val_acc: 0.7903
Epoch 4/15
4s - loss: 0.6037 - acc: 0.8174 - val_loss: 0.6724 - val_acc: 0.7986
Epoch 5/15
4s - loss: 0.5176 - acc: 0.8440 - val_loss: 0.6676 - val_acc: 0.8022
Epoch 6/15
4s - loss: 0.4370 - acc: 0.8683 - val_loss: 0.6796 - val_acc: 0.8029
Epoch 7/15
4s - loss: 0.3617 - acc: 0.8913 - val_loss: 0.7017 - val_acc: 0.8059
Epoch 8/15
4s - loss: 0.2876 - acc: 0.9140 - val_loss: 0.7405 - val_acc: 0.8062
Epoch 9/15
4s - loss: 0.2284 - acc: 0.9306 - val_loss: 0.7623 - val_acc: 0.8062
Epoch 10/15
4s - loss: 0.1728 - acc: 0.9466 - val_loss: 0.8177 - val_acc: 0.8064
Epoch 11/15
4s - loss: 0.1344 - acc: 0.9592 - val_loss: 0.8151 - val_acc: 0.8064
Epoch 12/15
4s - loss: 0.1043 - acc: 0.9684 - val_loss: 0.9129 - val_

In [9]:
#Now read the test json file in 
test = pd.read_json('../data/test.json/test.json')
test.head()

#Do the same thing we did with the training set and create a array using the count vectorizer. 
test_ingredients = test['ingredients']
test_ingredients_words = [' '.join(x) for x in test_ingredients]
test_ingredients_array = vectorizer.transform(test_ingredients_words).toarray()
test_tfidf = vectorizertfidf.fit_transform(test_ingredients_words).toarray()

In [10]:
print test_ingredients_array.shape
result = model.predict_classes(test_tfidf)
print type(result)
print result.shape

(9944, 2000)


ValueError: dimension mismatch in args to gemm (128,2246)x(3010,1024)->(128,1024)
Apply node that caused the error: GpuDot22(GpuFromHost.0, <CudaNdarrayType(float32, matrix)>)
Toposort index: 4
Inputs types: [CudaNdarrayType(float32, matrix), CudaNdarrayType(float32, matrix)]
Inputs shapes: [(128, 2246), (3010, 1024)]
Inputs strides: [(2246, 1), (1024, 1)]
Inputs values: ['not shown', 'not shown']
Outputs clients: [[GpuElemwise{Composite{(i0 * ((i1 + i2) + Abs((i1 + i2))))}}[(0, 1)](CudaNdarrayConstant{[[ 0.5]]}, GpuDot22.0, GpuDimShuffle{x,0}.0)]]

HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created. This can be done with by setting the Theano flag 'optimizer=fast_compile'. If that does not work, Theano optimizations can be disabled with 'optimizer=None'.
HINT: Use the Theano flag 'exception_verbosity=high' for a debugprint and storage map footprint of this apply node.

In [ ]:
result = le.inverse_transform(result)
print result

In [ ]:
# Copy the results to a pandas dataframe with an "id" column and
# a "cusine" column
output = pd.DataFrame( data={"id":test["id"], "cuisine":result} )

# Use pandas to write the comma-separated output file
output.to_csv( "ANN_model.csv", index=False, quoting=3 )